In [ ]:
import numpy as np
import random
from search.FCMBased.lingam import CAMUV
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from utils_PartDAG import sample_intervention_right, MLP, LR, IPS, Model_Multi
from causallearn.search.ConstraintBased.PC import pc
from demo_SCORE import *
from sklearn.linear_model import LogisticRegression

def factorial_sum(n):
    return 1 if n < 2 else n + factorial_sum(n - 1)


In [ ]:
def BFS_node(L, adj):
    visit = []
    while len(L) > 0:
        i = L[0]
        del L[0]    
        if i not in visit:
            visit.append(i)
            for j in np.nonzero(adj[i, :])[0]:
                if j not in visit and j not in L:
                    L.append(j)
    return visit

In [ ]:
def generate_graph(Graph):
    Graph_new_fair_relax = np.zeros_like(Graph)
    Graph_new_fair = np.zeros_like(Graph)
    Graph_new_ours = np.zeros_like(Graph)
    for i in range(Graph.shape[0]):
        for j in range(i + 1, Graph.shape[1]):
            if Graph[i, j] * Graph[j, i] > 0 and Graph[i, j] < 0: # undirected edge
                Graph_new_fair[i, j] = 1
                Graph_new_fair[j, i] = 1
                Graph_new_ours[i, j] = 2
                Graph_new_ours[j, i] = 2
            elif Graph[i, j] * Graph[j, i] < 0 and Graph[i, j] < 0: # i -> j
                Graph_new_fair_relax[i, j] = 1
                Graph_new_fair[i, j] = 1
                Graph_new_ours[i, j] = 1
            elif Graph[i, j] * Graph[j, i] > 0 and Graph[i, j] > 0: # double side direction
                Graph_new_fair_relax[i, j] = 1
                Graph_new_fair[i, j] = 1             
                Graph_new_ours[i, j] = 1
                Graph_new_fair_relax[j, i] = 1
                Graph_new_fair[j, i] = 1             
                Graph_new_ours[j, i] = 1                      
            elif Graph[i, j] * Graph[j, i] < 0 and Graph[i, j] > 0: # j -> i
                Graph_new_fair_relax[j, i] = 1
                Graph_new_fair[j, i] = 1 
                Graph_new_ours[j, i] = 1
    return Graph_new_fair, Graph_new_fair_relax, Graph_new_ours

In [ ]:
# 1.5

Oracle_RMSE = [[], [], [], []]

Unaware_RMSE = [[], [], [], []]
Unaware_fair = [[], [], [], []]   
Full_RMSE = [[], [], [], []]
Full_fair = [[], [], [], []]

Fair_RMSE = [[], [], [], []]
Fair_fair = [[], [], [], []]
Fair_relax_RMSE = [[], [], [], []]
Fair_relax_fair = [[], [], [], []]

Ours_RMSE = [[], [], [], []]
Ours_fair = [[], [], [], []]
for d in [10, 20, 30, 40]:
    if d == 20:
        print(d)
        print('Ours', np.mean(Ours_RMSE[0]), np.std(Ours_RMSE[0]), 
              np.mean(Ours_fair[0]), np.std(Ours_fair[0]))
        print('Oracle', np.mean(Oracle_RMSE[0]), np.std(Oracle_RMSE[0]), 0.0, 0.0)
        print('Unaware', np.mean(Unaware_RMSE[0]), np.std(Unaware_RMSE[0]), np.mean(Unaware_fair[0]), np.std(Unaware_fair[0]))
        print('Full', np.mean(Full_RMSE[0]), np.std(Full_RMSE[0]), np.mean(Full_fair[0]), np.std(Full_fair[0]))
        print('Fair', np.mean(Fair_RMSE[0]), np.std(Fair_RMSE[0]), np.mean(Fair_fair[0]), np.std(Fair_fair[0]))
        print('Fair_relax', np.mean(Fair_relax_RMSE[0]), np.std(Fair_relax_RMSE[0]), np.mean(Fair_relax_fair[0]), np.std(Fair_relax_fair[0]))
        
    if d == 30:
        print(d)
        print('Ours', np.mean(Ours_RMSE[1]), np.std(Ours_RMSE[1]), 
              np.mean(Ours_fair[1]), np.std(Ours_fair[1]))        
        print('Oracle', np.mean(Oracle_RMSE[1]), np.std(Oracle_RMSE[1]), 0.0, 0.0)
        print('Unaware', np.mean(Unaware_RMSE[1]), np.std(Unaware_RMSE[1]), np.mean(Unaware_fair[1]), np.std(Unaware_fair[1]))
        print('Full', np.mean(Full_RMSE[1]), np.std(Full_RMSE[1]), np.mean(Full_fair[1]), np.std(Full_fair[1]))
        print('Fair', np.mean(Fair_RMSE[1]), np.std(Fair_RMSE[1]), np.mean(Fair_fair[1]), np.std(Fair_fair[1]))
        print('Fair_relax', np.mean(Fair_relax_RMSE[1]), np.std(Fair_relax_RMSE[1]), np.mean(Fair_relax_fair[1]), np.std(Fair_relax_fair[1]))
        
    if d == 40:
        print(d)
        print('Ours', np.mean(Ours_RMSE[2]), np.std(Ours_RMSE[2]), 
              np.mean(Ours_fair[2]), np.std(Ours_fair[2]))
        print('Oracle', np.mean(Oracle_RMSE[2]), np.std(Oracle_RMSE[2]), 0.0, 0.0)
        print('Unaware', np.mean(Unaware_RMSE[2]), np.std(Unaware_RMSE[2]), np.mean(Unaware_fair[2]), np.std(Unaware_fair[2]))
        print('Full', np.mean(Full_RMSE[2]), np.std(Full_RMSE[2]), np.mean(Full_fair[2]), np.std(Full_fair[2]))
        print('Fair', np.mean(Fair_RMSE[2]), np.std(Fair_RMSE[2]), np.mean(Fair_fair[2]), np.std(Fair_fair[2]))
        print('Fair_relax', np.mean(Fair_relax_RMSE[2]), np.std(Fair_relax_RMSE[2]), np.mean(Fair_relax_fair[2]), np.std(Fair_relax_fair[2]))         
    node_names = list(np.arange(d))        

    
    for num in range(50):
        X, adj, noise, beta, Y, node = generate_right(d, 2*d, 1000, noise_std = 1.5, GP=True)
        X = np.array(X)
        cg = pc(X, alpha = 0.05, node_names = node_names)
        
        L = list(np.nonzero(adj[node, :])[0])
        visit = BFS_node(L, adj)
        visit.append(node)
        
        inter_value = (1 - X[:, node])
        Inter_X = sample_intervention_right(d = d, n =1000, noise = noise, beta = beta, inter_value = inter_value, node = node, Y = Y, adj = adj)

        oracle_node = np.array(list(set(np.arange(d)) - set(visit) - set({Y}) - set({node})))
        full_node = np.array(list(set(np.arange(d)) - set({Y})))
        unaware_node = np.array(list(set(np.arange(d)) - set({node}) - set({Y})))

        Inter_X = np.array(Inter_X)
        total_X = np.c_[X, Inter_X]
        node_Y = X[:, Y]
        X_train, X_test, Y_train, Y_test = train_test_split(total_X, node_Y, test_size = 0.2, random_state = 0)
        X_train = X_train[:, :X.shape[1]]
        X_test_inter = X_test[:, X.shape[1]:].copy()
        X_test = X_test[:, :X.shape[1]]

        X_train_del = np.delete(X_train, [node, Y], 1)
        X_test_del = np.delete(X_test, [node, Y], 1)
        X_test_inter_del = np.delete(X_test_inter, [node, Y], 1)

        if len(oracle_node) == 0:
            test_pred_oracle = 0
        else:
            oracle_X_train = X_train[:, oracle_node]
            oracle_X_test = X_test[:, oracle_node]
            lm_oracle = linear_model.LinearRegression()
            lm_oracle.fit(oracle_X_train, Y_train)
            test_pred_oracle  = lm_oracle.predict(oracle_X_test)

        Oracle_RMSE[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_oracle-Y_test)**2)))        
        
        full_X_train = X_train[:, full_node]
        full_X_test = X_test[:, full_node]
        lm_full = linear_model.LinearRegression()
        lm_full.fit(full_X_train, Y_train)
        test_pred_full  = lm_full.predict(full_X_test)

        Full_RMSE[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_full-Y_test)**2)))

        full_X_train_inter = X_test_inter[:, full_node]
        test_pred_full_inter = lm_full.predict(full_X_train_inter)
        Full_fair[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_full-test_pred_full_inter)**2)))        
            
        unaware_X_train = X_train[:, unaware_node]
        unaware_X_test = X_test[:, unaware_node]
        lm_unaware = linear_model.LinearRegression()
        lm_unaware.fit(unaware_X_train, Y_train)
        test_pred_unaware  = lm_unaware.predict(unaware_X_test)

        Unaware_RMSE[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_unaware-Y_test)**2)))

        unaware_X_train_inter = X_test_inter[:, unaware_node]
        test_pred_unaware_inter = lm_unaware.predict(unaware_X_train_inter) 
        Unaware_fair[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_unaware-test_pred_unaware_inter)**2)))             
            
        visit_fair_relax = []
        visit_fair = []
        Graph = cg.G.graph
        Graph_new_fair, Graph_new_fair_relax, Graph_new_ours = generate_graph(Graph)

        List_2 = list(np.nonzero(Graph_new_ours[:, node] == 2)[0]) # possible parent node
        List_1 = list(np.nonzero(Graph_new_ours[:, node] == 1)[0]) # exist parent node
        print('exist parent node', List_1, 'possible parent node', List_2)
        possible_backdoor = []
        if List_1 == []:
            possible_backdoor = List_2.copy()
        elif List_2 == []:
            possible_backdoor = List_1
        else:
            for i in List_2:
                for j in range(len(List_1)):
                    if i in np.nonzero(Graph_new_ours[List_1[j], :])[0] or i in np.nonzero(Graph_new_ours[:, List_1[j]])[0]:
                        if j == len(List_1) - 1:
                            possible_backdoor.append(i)
                    else:
                        break

        if len(possible_backdoor) > 1 and possible_backdoor != List_1:
            N = len(possible_backdoor)
            ALL_possible = []
            for i in range(2 ** N):
                combo = []
                for j in range(N):
                    if(i>>j)%2:
                        combo.append(possible_backdoor[j])
                ALL_possible.append(combo)

            for i in ALL_possible:
                if len(i) > 1:
                    temp = Graph_new_ours[:, i]
                    temp = temp[i, :]
                    if (len(np.nonzero(np.triu(temp))[0]) == factorial_sum(len(i)) - len(i)) or (len(np.nonzero(np.tril(temp))[0]) == factorial_sum(len(i)) - len(i)):
                        possible_backdoor.append(i)                 

        if possible_backdoor == List_1 and List_1 != []:
            possible_backdoor = [possible_backdoor]
        else:
            for i in range(len(possible_backdoor)):
                if type(possible_backdoor[i]) is not list:
                    possible_backdoor[i] = [possible_backdoor[i]]
                    
        if List_1 != [] and possible_backdoor != [List_1]:
            for i in possible_backdoor:
                i.extend(List_1)
            possible_backdoor.insert(0, List_1)

        print('possible set', possible_backdoor) # possible set
        print('real set', np.nonzero(adj[:, node])) # real set

        fair_relax = list(np.nonzero(Graph_new_fair_relax[node, :])[0])    
        fair = list(np.nonzero(Graph_new_fair[node, :])[0])

        visit_fair_relax = BFS_node(fair_relax, Graph_new_fair_relax)
        visit_fair_relax.append(node)
        visit_fair = BFS_node(fair, Graph_new_fair)
        visit_fair.append(node)

        fair_node = np.array(list(set(np.arange(d)) - set(visit_fair) - set({Y}) - set({node})))
        fair_relax_node = np.array(list(set(np.arange(d)) - set(visit_fair_relax) - set({Y}) - set({node})))

        propensity = np.zeros(X_train.shape[0])

        if possible_backdoor == []: 
            propensity[X_train[:, node] == 0] = np.mean(X_train[:, node] == 0)
            propensity[X_train[:, node] == 1] = np.mean(X_train[:, node] == 1)   
        else:    
            j = 0
            for i in possible_backdoor:
                prop_estimate_model = MLP(X_train[:, i].reshape(X_train.shape[0], -1), X_train[:, node])
                prop_estimate_model.cuda()
                prop_estimate_model.fit(lamb = 1e-4)
                pred = prop_estimate_model.predict(X_train[:, i].reshape(X_train.shape[0], -1))
                if j == 0:
                    j += 1
                    propensity = pred
                else:
                    j += 1
                    propensity = np.c_[propensity, pred]

        propensity = np.clip(propensity.reshape(X_train.shape[0], -1), 0.2, 0.8)

        if len(fair_node) == 0:
            test_pred_fair = 0
        else:
            fair_X_train = X_train[:, fair_node]
            fair_X_test = X_test[:, fair_node]
            lm_fair = linear_model.LinearRegression()
            lm_fair.fit(fair_X_train, Y_train)
            test_pred_fair  = lm_fair.predict(fair_X_test)

        Fair_RMSE[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_fair-Y_test)**2)))

        if len(fair_relax_node) == 0:
            test_pred_fair_relax = 0
        else:
            fair_relax_X_train = X_train[:, fair_relax_node]
            fair_relax_X_test = X_test[:, fair_relax_node]
            lm_fair_relax = linear_model.LinearRegression()
            lm_fair_relax.fit(fair_relax_X_train, Y_train)
            test_pred_fair_relax  = lm_fair_relax.predict(fair_relax_X_test)

        Fair_relax_RMSE[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_fair_relax-Y_test)**2)))            

        if len(fair_node) == 0:
            test_pred_fair_inter = 0
            Fair_fair[int(d/10 - 1)].append(0)
        else:
            fair_X_train_inter = X_test_inter[:, fair_node]
            test_pred_fair_inter = lm_fair.predict(fair_X_train_inter)

            Fair_fair[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_fair-test_pred_fair_inter)**2)))

        if len(fair_relax_node) == 0:
            test_pred_fair_relax_inter = 0
            Fair_relax_fair[int(d/10 - 1)].append(0)
        else:
            fair_relax_X_train_inter = X_test_inter[:, fair_relax_node]
            test_pred_fair_relax_inter = lm_fair_relax.predict(fair_relax_X_train_inter)
     
            Fair_relax_fair[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_fair_relax-test_pred_fair_relax_inter)**2)))            

            model = Model_Multi(X_train_del, Y_train, X_train[:, node], propensity, output_dim = propensity.shape[1])
            model.cuda()
            model.fit(constrain = 0.2, alpha = 15, C = 0.03, lr = 0.01)

            test_pred_ours  = model.LR.predict(X_test_del)

            Ours_RMSE[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_ours-Y_test)**2)))

            test_pred_ours_inter = model.LR.predict(X_test_inter_del)

            Ours_fair[int(d/10 - 1)].append(np.sqrt(np.mean((test_pred_ours-test_pred_ours_inter)**2)))    
print(d)
print('Ours', np.mean(Ours_RMSE[3]), np.std(Ours_RMSE[3]), 
      np.mean(Ours_fair[3]), np.std(Ours_fair[3]))
print('Oracle', np.mean(Oracle_RMSE[3]), np.std(Oracle_RMSE[3]), 0.0, 0.0)
print('Unaware', np.mean(Unaware_RMSE[3]), np.std(Unaware_RMSE[3]), np.mean(Unaware_fair[3]), np.std(Unaware_fair[3]))
print('Full', np.mean(Full_RMSE[3]), np.std(Full_RMSE[3]), np.mean(Full_fair[3]), np.std(Full_fair[3]))
print('Fair', np.mean(Fair_RMSE[3]), np.std(Fair_RMSE[3]), np.mean(Fair_fair[3]), np.std(Fair_fair[3]))
print('Fair_relax', np.mean(Fair_relax_RMSE[3]), np.std(Fair_relax_RMSE[3]), np.mean(Fair_relax_fair[3]), np.std(Fair_relax_fair[3]))                        